# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'True'

import tequila as tq
from utility import *

First, we set up the Hamiltonian in Tequila's format and the unitary gates obtained in Step 3. 

In [2]:
H = tq.QubitHamiltonian.from_openfermion(get_qubit_hamiltonian('h2', 2, 'sto-3g', qubit_transf='jw'))

a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4)
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(1)X(2)X(3)"), angle=a)
print(U)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 1, 2, 3), control=(), parameter=tau_0, paulistring=X(0)Y(1)X(2)X(3))



One can check the expectation value to see it is near the ground state energy.

In [3]:
E = tq.ExpectationValue(H=H, U=U)
vars = {'beta_1': 3.141592624143881, 'beta_0': 3.141592624143881, 'tau_0': 1.1331410014096885, 'gamma_1': 0.0, 'beta_3': 0.0, 'gamma_3': 0.0, 'gamma_2': 0.0, 'gamma_0': 0.0, 'beta_2': 0.0} # values obtained from step 3
print(tq.simulate(E, variables=vars))

-0.9486411121761623


One can run the same experiment on a real quantum computer through IBM Quantum Experience (ibmq). After activating your account here (https://quantum-computing.ibm.com/login), copy the API token and execute the commented block below. 

In [4]:
from qiskit import IBMQ
token = 'ec714f60c3eed4a2b8baba21ca2bf304758985a4e85ad35646be1f41012596b94236ec1ce5eca31eeaf49179a40c68a6ad85fbfa3ec3c0d9318997ecd9c89d60'
# hub='ibm-q-kaist'
# group='internal'
# project='reservations'
IBMQ.save_account(token, overwrite=True)
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_manila')
# pr = IBMQ.get_provider(hub=hub, group=group, project=project)
# print(pr)
print(provider, backend)

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')> ibmq_manila


In [5]:
# list of devices available can be found in ibmq account page
# Try a device that isn't too busy. If the wait time is too long, measure only one pauli-word in the H2 Hamilotonian using the commented line
E = tq.ExpectationValue(H=tq.QubitHamiltonian.from_paulistrings(
    [PauliString({ 0:'X', 1:'X', 2:'Y', 3:'Y' }, 1)]), U=U)
tq.simulate(E, variables=vars, samples=100, backend="qiskit", device='ibmq_manila')

0.54

![simulation_result.png](https://raw.githubusercontent.com/QuNovaComputing/CohortProject_2021/week3_qunovacomputing/Week3_VQE/figures/Screenshot%20from%202021-07-23%2022-55-25.png)


The following code block prints the circuit.

In [ ]:
circ = tq.circuit.compiler.compile_exponential_pauli_gate(U)
tq.draw(circ, backend="qiskit")

